**1. ติดตั้งไลบรารีที่ใช้**

In [1]:
!pip install -q roboflow tensorflow scikit-learn

**2. import ไลบรารี**

In [2]:
import os
import numpy as np

from roboflow import Roboflow

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.metrics import classification_report, confusion_matrix

**3. โหลด Dataset จาก Roboflow (ใช้ API)**

In [3]:
ROBOFLOW_API_KEY = "CsQXP0pyvTdf7j7C3j1k"

WORKSPACE = "areeyamay"
PROJECT   = "meowmood-udr3r"
VERSION   = 2

rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace(WORKSPACE).project(PROJECT)
dataset = project.version(VERSION).download("folder")

base_dir = dataset.location
print("Dataset base dir:", base_dir)

loading Roboflow workspace...
loading Roboflow project...
Exporting format folder in progress : 85.0%
Version export complete for folder format



Extracting Dataset Version Zip to MeowMood-2 in folder:: 100%|██████████| 5668/5668 [00:01<00:00, 3390.54it/s]

Dataset base dir: /content/MeowMood-2


**4. เตรียม train/valid/test และ DataGenerator**

In [8]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_dir = os.path.join(base_dir, "train")
valid_dir = os.path.join(base_dir, "valid")
test_dir  = os.path.join(base_dir, "test")

print("Train dir:", train_dir)
print("Valid dir:", valid_dir)
print("Test dir :", test_dir)

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode="nearest"
)

valid_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

valid_gen = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

print("class_indices:", train_gen.class_indices)

Train dir: /content/MeowMood-2/train
Valid dir: /content/MeowMood-2/valid
Test dir : /content/MeowMood-2/test
Found 4944 images belonging to 4 classes.
Found 472 images belonging to 4 classes.
Found 235 images belonging to 4 classes.
class_indices: {'angry': 0, 'neutral': 1, 'relaxed': 2, 'scared': 3}


**5. สร้างโมเดล ResNet50 (Transfer Learning – Phase 1)**

In [9]:
NUM_CLASSES = train_gen.num_classes
print("NUM_CLASSES:", NUM_CLASSES)

base_model = ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)

for layer in base_model.layers:
    layer.trainable = False

inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = models.Model(inputs, outputs)

loss_fn = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss=loss_fn,
    metrics=["accuracy"]
)

model.summary()

NUM_CLASSES: 4


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 4)              │         1,028 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,113,284 (91.98 MB)

 Trainable params: 525,572 (2.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

**6. เทรน Phase 1 (ฝึกเฉพาะหัวบน ๆ)**

In [10]:
checkpoint1 = "best_phase1.h5"

callbacks1 = [
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    ModelCheckpoint(checkpoint1, monitor="val_accuracy", save_best_only=True)
]

EPOCHS_PHASE1 = 15

history1 = model.fit(
    train_gen,
    validation_data=valid_gen,
    epochs=EPOCHS_PHASE1,
    callbacks=callbacks1
)

Epoch 1/15
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.3612 - loss: 1.6053

155/155 ━━━━━━━━━━━━━━━━━━━━ 912s 6s/step - accuracy: 0.3619 - loss: 1.6036 - val_accuracy: 0.6695 - val_loss: 0.9873
Epoch 2/15
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.6097 - loss: 1.0585

155/155 ━━━━━━━━━━━━━━━━━━━━ 916s 6s/step - accuracy: 0.6099 - loss: 1.0582 - val_accuracy: 0.6992 - val_loss: 0.9076
Epoch 3/15
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.6828 - loss: 0.9434

155/155 ━━━━━━━━━━━━━━━━━━━━ 902s 6s/step - accuracy: 0.6829 - loss: 0.9433 - val_accuracy: 0.7225 - val_loss: 0.8691
Epoch 4/15
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.7321 - loss: 0.8718

155/155 ━━━━━━━━━━━━━━━━━━━━ 903s 6s/step - accuracy: 0.7321 - loss: 0.8718 - val_accuracy: 0.7479 - val_loss: 0.8366
Epoch 5/15
155/155 ━━━━━━━━━━━━━━━━━━━━ 919s 6s/step - accuracy: 0.7540 - loss: 0.8315 - val_accuracy: 0.7479 - val_loss: 0.8300
Epoch 6/15
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.7836 - loss: 0.7780

155/155 ━━━━━━━━━━━━━━━━━━━━ 910s 6s/step - accuracy: 0.7836 - loss: 0.7780 - val_accuracy: 0.7521 - val_loss: 0.8179
Epoch 7/15
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.8059 - loss: 0.7439

155/155 ━━━━━━━━━━━━━━━━━━━━ 911s 6s/step - accuracy: 0.8059 - loss: 0.7439 - val_accuracy: 0.7542 - val_loss: 0.8105
Epoch 8/15
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.8203 - loss: 0.7272

155/155 ━━━━━━━━━━━━━━━━━━━━ 912s 6s/step - accuracy: 0.8203 - loss: 0.7272 - val_accuracy: 0.7775 - val_loss: 0.7962
Epoch 9/15
155/155 ━━━━━━━━━━━━━━━━━━━━ 902s 6s/step - accuracy: 0.8373 - loss: 0.7013 - val_accuracy: 0.7775 - val_loss: 0.8025
Epoch 10/15
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.8480 - loss: 0.6863

155/155 ━━━━━━━━━━━━━━━━━━━━ 924s 6s/step - accuracy: 0.8480 - loss: 0.6863 - val_accuracy: 0.7860 - val_loss: 0.7933
Epoch 11/15
155/155 ━━━━━━━━━━━━━━━━━━━━ 920s 6s/step - accuracy: 0.8746 - loss: 0.6456 - val_accuracy: 0.7754 - val_loss: 0.7895
Epoch 12/15
155/155 ━━━━━━━━━━━━━━━━━━━━ 909s 6s/step - accuracy: 0.8688 - loss: 0.6444 - val_accuracy: 0.7754 - val_loss: 0.7947
Epoch 13/15
155/155 ━━━━━━━━━━━━━━━━━━━━ 898s 6s/step - accuracy: 0.8831 - loss: 0.6241 - val_accuracy: 0.7818 - val_loss: 0.7856
Epoch 14/15
155/155 ━━━━━━━━━━━━━━━━━━━━ 898s 6s/step - accuracy: 0.8830 - loss: 0.6218 - val_accuracy: 0.7754 - val_loss: 0.7722
Epoch 15/15
155/155 ━━━━━━━━━━━━━━━━━━━━ 899s 6s/step - accuracy: 0.8934 - loss: 0.6064 - val_accuracy: 0.7818 - val_loss: 0.7726


**7. Fine-tune Phase 2 (เปิดบางชั้นของ ResNet50 ให้เรียนเพิ่ม)**

In [11]:
model.load_weights(checkpoint1)

for layer in base_model.layers[-20:]:
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

checkpoint2 = "best_phase2.h5"

callbacks2 = [
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    ModelCheckpoint(checkpoint2, monitor="val_accuracy", save_best_only=True)
]

EPOCHS_PHASE2 = 15

history2 = model.fit(
    train_gen,
    validation_data=valid_gen,
    epochs=EPOCHS_PHASE2,
    callbacks=callbacks2
)

Epoch 1/15
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.8695 - loss: 0.4634

155/155 ━━━━━━━━━━━━━━━━━━━━ 1182s 8s/step - accuracy: 0.8694 - loss: 0.4634 - val_accuracy: 0.7924 - val_loss: 0.5508
Epoch 2/15
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.8794 - loss: 0.3726

155/155 ━━━━━━━━━━━━━━━━━━━━ 1178s 8s/step - accuracy: 0.8794 - loss: 0.3725 - val_accuracy: 0.8008 - val_loss: 0.5368
Epoch 3/15
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.8968 - loss: 0.3151

155/155 ━━━━━━━━━━━━━━━━━━━━ 1161s 7s/step - accuracy: 0.8968 - loss: 0.3151 - val_accuracy: 0.8093 - val_loss: 0.5225
Epoch 4/15
155/155 ━━━━━━━━━━━━━━━━━━━━ 1162s 8s/step - accuracy: 0.9065 - loss: 0.2781 - val_accuracy: 0.8030 - val_loss: 0.5181
Epoch 5/15
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.9260 - loss: 0.2405

155/155 ━━━━━━━━━━━━━━━━━━━━ 1168s 8s/step - accuracy: 0.9260 - loss: 0.2404 - val_accuracy: 0.8136 - val_loss: 0.5089
Epoch 6/15
155/155 ━━━━━━━━━━━━━━━━━━━━ 1173s 8s/step - accuracy: 0.9396 - loss: 0.2009 - val_accuracy: 0.8072 - val_loss: 0.5170
Epoch 7/15
155/155 ━━━━━━━━━━━━━━━━━━━━ 1224s 8s/step - accuracy: 0.9440 - loss: 0.1868 - val_accuracy: 0.8072 - val_loss: 0.5229
Epoch 8/15
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.9570 - loss: 0.1556

155/155 ━━━━━━━━━━━━━━━━━━━━ 1171s 8s/step - accuracy: 0.9570 - loss: 0.1556 - val_accuracy: 0.8263 - val_loss: 0.5274
Epoch 9/15
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.9527 - loss: 0.1490

155/155 ━━━━━━━━━━━━━━━━━━━━ 1166s 8s/step - accuracy: 0.9528 - loss: 0.1489 - val_accuracy: 0.8305 - val_loss: 0.5450
Epoch 10/15
155/155 ━━━━━━━━━━━━━━━━━━━━ 1223s 8s/step - accuracy: 0.9700 - loss: 0.1210 - val_accuracy: 0.8242 - val_loss: 0.5487


**8. ประเมินผลบน Test Set + ดู Accuracy**

In [12]:
model.load_weights(checkpoint2)

test_loss, test_acc = model.evaluate(test_gen)
print("✅ Test Loss:", test_loss)
print("✅ Test Accuracy:", test_acc)

8/8 ━━━━━━━━━━━━━━━━━━━━ 38s 5s/step - accuracy: 0.7606 - loss: 0.7341
✅ Test Loss: 0.7159852385520935
✅ Test Accuracy: 0.800000011920929


**9. รายงานละเอียดเพิ่ม (classification report + confusion matrix)**

In [13]:
y_pred_proba = model.predict(test_gen)
y_pred = np.argmax(y_pred_proba, axis=1)
y_true = test_gen.classes

class_names = list(test_gen.class_indices.keys())
print("Class names:", class_names)

print(classification_report(y_true, y_pred, target_names=class_names))

cm = confusion_matrix(y_true, y_pred)
print("Confusion matrix:\n", cm)

8/8 ━━━━━━━━━━━━━━━━━━━━ 45s 5s/step
Class names: ['angry', 'neutral', 'relaxed', 'scared']
              precision    recall  f1-score   support

       angry       0.81      0.75      0.78        57
     neutral       0.71      0.71      0.71        59
     relaxed       0.80      0.83      0.82        64
      scared       0.88      0.91      0.89        55

    accuracy                           0.80       235
   macro avg       0.80      0.80      0.80       235
weighted avg       0.80      0.80      0.80       235

Confusion matrix:
 [[43  6  5  3]
 [ 6 42  7  4]
 [ 2  9 53  0]
 [ 2  2  1 50]]


**10. เซฟโมเดลเป็นไฟล์ .h5**

In [14]:
model.save("meowmood_model.h5")
print("Saved model as meowmood_model.h5")

Saved model as meowmood_model.h5
